In [20]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras

import cv2
import numpy as np

In [21]:
# Load the TrackNet with pretrained weights
from tracknet import trackNet
model = trackNet(256, input_height=360, input_width=640)
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.load_weights('model.1')

layer24 output shape: 256 360 640
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3, 360, 640)]     0         
                                                                 
 conv2d_54 (Conv2D)          (None, 64, 360, 640)      1792      
                                                                 
 activation_57 (Activation)  (None, 64, 360, 640)      0         
                                                                 
 batch_normalization_54 (Ba  (None, 64, 360, 640)      2560      
 tchNormalization)                                               
                                                                 
 conv2d_55 (Conv2D)          (None, 64, 360, 640)      36928     
                                                                 
 activation_58 (Activation)  (None, 64, 360, 640)      0         
                         

2023-12-14 01:12:13.752020: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open model.1: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


In [27]:
# Track and annotate the ball on a video
cap = cv2.VideoCapture('data/videos/video1.mp4')

# skip the first 20000 frames
cap.set(cv2.CAP_PROP_POS_FRAMES, 55000)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (640, 360))

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # permute to CHW
    rgb = np.transpose(rgb, (2, 0, 1))

    # Make prediction
    pred = model.predict(np.expand_dims(rgb, axis=0))[0]

    pred = pred.reshape((360, 640, 256)).argmax(axis=2)

    # cv2 image must be numpy.uint8, convert numpy.int64 to numpy.uint8
    pred = pred.astype(np.uint8)

    # Draw the ball
    if np.sum(pred) > 0:
        x, y = np.where(pred == 1)
        frame = cv2.circle(frame, (y[0], x[0]), 10, (0, 255, 0), -1)

    # Show the frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 71ms/step


IndexError: index 0 is out of bounds for axis 0 with size 0

: 